In [4]:
from farmdataset import FarmDataset
import torch as tc
import gdal
from torchvision import transforms
import png
import numpy as np

In [9]:
use_cuda=True
model=tc.load('./tmp/model20')  #torch.save(model,'./tmp/model{}'.format(epoch))
device = tc.device("cuda" if use_cuda else "cpu")
model=model.to(device)
model.eval()
ds=FarmDataset(istrain=False)

def createres(d,outputname):
    #创建一个和ds大小相同的灰度图像BMP
    driver = gdal.GetDriverByName("BMP")
    #driver=ds.GetDriver()
    od=driver.Create('./tmp/'+outputname,d.RasterXSize,d.RasterYSize,1)
    return od
 
def createpng(height,width,data,outputname):
    w=png.Writer(width,height,bitdepth=2,greyscale=True)
    of=open('./tmp/'+outputname,'wb')
    w.write_array(of,data.flat)
    of.close()
    return

def predict(d,outputname='tmp.bmp'):
    wx=d.RasterXSize   #width
    wy=d.RasterYSize   #height
    print(wx,wy)
    od=data=np.zeros((wy,wx),np.uint8)
    #od=createres(d,outputname=outputname)
    #ob=od.GetRasterBand(1) #得到第一个channnel
    blocksize = 512
    step = 512
    for cy in range(step,wy-blocksize,step):
        for cx in range(step,wx-blocksize,step):
            img=d.ReadAsArray(cx-step,cy-step,blocksize,blocksize)[0:3,:,:] #channel*h*w
            if (img.sum()==0): continue
            x=tc.from_numpy(img/255.0).float()
            #print(x.shape)
            x=x.unsqueeze(0).to(device)
            r=model.forward(x)
            r=tc.argmax(r.cpu()[0],0).byte().numpy()  #512*512
#             print(r)
            #ob.WriteArray(r,cx,cy)
#             od[cy-step//2:cy+step//2,cx-step//2:cx+step//2]=r[256:step+256,256:step+256]
            od[cy-step:cy, cx-step:cx] = r
            print(cy,cx)
    #del od
    createpng(wy,wx,od,outputname)
    return

In [10]:
print("start predict.....")
predict(ds[0],'image_3_predict.png')
print("start predict 2 .....")
predict(ds[1],'image_4_predict.png')

start predict.....
37241 19903
512 13824
512 14336
512 14848
512 15360
512 15872
512 16384
512 16896
512 17408
512 17920
512 18432
512 18944
512 19456
512 19968
512 20480
512 20992
512 21504
512 22016
512 22528
512 23040
1024 13312
1024 13824
1024 14336
1024 14848
1024 15360
1024 15872
1024 16384
1024 16896
1024 17408
1024 17920
1024 18432
1024 18944
1024 19456
1024 19968
1024 20480
1024 20992
1024 21504
1024 22016
1024 22528
1024 23040
1024 23552
1536 12288
1536 12800
1536 13312
1536 13824
1536 14336
1536 14848
1536 15360
1536 15872
1536 16384
1536 16896
1536 17408
1536 17920
1536 18432
1536 18944
1536 19456
1536 19968
1536 20480
1536 20992
1536 21504
1536 22016
1536 22528
1536 23040
1536 23552
1536 24064
1536 24576
2048 11776
2048 12288
2048 12800
2048 13312
2048 13824
2048 14336
2048 14848
2048 15360
2048 15872
2048 16384
2048 16896
2048 17408
2048 17920
2048 18432
2048 18944
2048 19456
2048 19968
2048 20480
2048 20992
2048 21504
2048 22016
2048 22528
2048 23040
2048 23552
2048 2406

9216 19968
9216 20480
9216 20992
9216 21504
9216 22016
9216 22528
9216 23040
9216 23552
9216 24064
9216 24576
9216 25088
9216 25600
9216 26112
9216 26624
9216 27136
9216 27648
9216 28160
9216 28672
9216 29184
9216 29696
9216 30208
9216 30720
9216 31232
9216 31744
9216 32256
9216 32768
9216 33280
9216 33792
9216 34304
9216 34816
9216 35328
9216 35840
9216 36352
9728 1536
9728 2048
9728 2560
9728 3072
9728 3584
9728 4096
9728 4608
9728 5120
9728 5632
9728 6144
9728 6656
9728 7168
9728 7680
9728 8192
9728 8704
9728 9216
9728 9728
9728 10240
9728 10752
9728 11264
9728 11776
9728 12288
9728 12800
9728 13312
9728 13824
9728 14336
9728 14848
9728 15360
9728 15872
9728 16384
9728 16896
9728 17408
9728 17920
9728 18432
9728 18944
9728 19456
9728 19968
9728 20480
9728 20992
9728 21504
9728 22016
9728 22528
9728 23040
9728 23552
9728 24064
9728 24576
9728 25088
9728 25600
9728 26112
9728 26624
9728 27136
9728 27648
9728 28160
9728 28672
9728 29184
9728 29696
9728 30208
9728 30720
9728 31232
9728 

14336 22016
14336 22528
14336 23040
14336 23552
14336 24064
14336 24576
14336 25088
14336 25600
14336 26112
14336 26624
14336 27136
14336 27648
14336 28160
14336 28672
14336 29184
14336 29696
14336 30208
14336 30720
14336 31232
14336 31744
14336 32256
14336 32768
14336 33280
14336 33792
14336 34304
14336 34816
14336 35328
14336 35840
14336 36352
14848 512
14848 1024
14848 1536
14848 2048
14848 2560
14848 3072
14848 3584
14848 4096
14848 4608
14848 5120
14848 5632
14848 6144
14848 6656
14848 7168
14848 7680
14848 8192
14848 8704
14848 9216
14848 9728
14848 10240
14848 10752
14848 11264
14848 11776
14848 12288
14848 12800
14848 13312
14848 13824
14848 14336
14848 14848
14848 15360
14848 15872
14848 16384
14848 16896
14848 17408
14848 17920
14848 18432
14848 18944
14848 19456
14848 19968
14848 20480
14848 20992
14848 21504
14848 22016
14848 22528
14848 23040
14848 23552
14848 24064
14848 24576
14848 25088
14848 25600
14848 26112
14848 26624
14848 27136
14848 27648
14848 28160
14848 28672


3072 7680
3072 8192
3072 8704
3072 9216
3072 9728
3072 10240
3072 10752
3072 11264
3072 11776
3072 12288
3072 12800
3584 512
3584 1024
3584 1536
3584 2048
3584 2560
3584 3072
3584 3584
3584 4096
3584 4608
3584 5120
3584 5632
3584 6144
3584 6656
3584 7168
3584 7680
3584 8192
3584 8704
3584 9216
3584 9728
3584 10240
3584 10752
3584 11264
3584 11776
3584 12288
3584 12800
3584 13312
4096 512
4096 1024
4096 1536
4096 2048
4096 2560
4096 3072
4096 3584
4096 4096
4096 4608
4096 5120
4096 5632
4096 6144
4096 6656
4096 7168
4096 7680
4096 8192
4096 8704
4096 9216
4096 9728
4096 10240
4096 10752
4096 11264
4096 11776
4096 12288
4096 12800
4096 13312
4608 512
4608 1024
4608 1536
4608 2048
4608 2560
4608 3072
4608 3584
4608 4096
4608 4608
4608 5120
4608 5632
4608 6144
4608 6656
4608 7168
4608 7680
4608 8192
4608 8704
4608 9216
4608 9728
4608 10240
4608 10752
4608 11264
4608 11776
4608 12288
4608 12800
4608 13312
5120 512
5120 1024
5120 1536
5120 2048
5120 2560
5120 3072
5120 3584
5120 4096
5120 46

13824 16384
13824 16896
13824 17408
13824 17920
13824 18432
13824 18944
13824 19456
13824 19968
13824 20480
13824 20992
13824 21504
13824 22016
13824 22528
13824 23040
13824 23552
13824 24064
13824 24576
13824 25088
14336 2048
14336 2560
14336 3072
14336 3584
14336 4096
14336 4608
14336 5120
14336 5632
14336 6144
14336 6656
14336 7168
14336 7680
14336 8192
14336 8704
14336 9216
14336 9728
14336 10240
14336 10752
14336 11264
14336 11776
14336 12288
14336 12800
14336 13312
14336 13824
14336 14336
14336 14848
14336 15360
14336 15872
14336 16384
14336 16896
14336 17408
14336 17920
14336 18432
14336 18944
14336 19456
14336 19968
14336 20480
14336 20992
14336 21504
14336 22016
14336 22528
14336 23040
14336 23552
14336 24064
14336 24576
14336 25088
14848 2048
14848 2560
14848 3072
14848 3584
14848 4096
14848 4608
14848 5120
14848 5632
14848 6144
14848 6656
14848 7168
14848 7680
14848 8192
14848 8704
14848 9216
14848 9728
14848 10240
14848 10752
14848 11264
14848 11776
14848 12288
14848 12800


21504 11264
21504 11776
21504 12288
21504 12800
21504 13312
21504 13824
21504 14336
21504 14848
21504 15360
21504 15872
21504 16384
21504 16896
21504 17408
21504 17920
21504 18432
21504 18944
21504 19456
21504 19968
21504 20480
21504 20992
21504 21504
21504 22016
21504 22528
21504 23040
21504 23552
21504 24064
21504 24576
22016 512
22016 1024
22016 1536
22016 2048
22016 2560
22016 3072
22016 3584
22016 4096
22016 4608
22016 5120
22016 5632
22016 6144
22016 6656
22016 7168
22016 7680
22016 8192
22016 8704
22016 9216
22016 9728
22016 10240
22016 10752
22016 11264
22016 11776
22016 12288
22016 12800
22016 13312
22016 13824
22016 14336
22016 14848
22016 15360
22016 15872
22016 16384
22016 16896
22016 17408
22016 17920
22016 18432
22016 18944
22016 19456
22016 19968
22016 20480
22016 20992
22016 21504
22016 22016
22016 22528
22016 23040
22016 23552
22016 24064
22016 24576
22528 1024
22528 1536
22528 2048
22528 2560
22528 3072
22528 3584
22528 4096
22528 4608
22528 5120
22528 5632
22528 6144